# Necessary Imports

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D

# Defining the model

In [2]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

# Implementing the Model

In [10]:
#load the trained model
model.load_weights('model.h5')

# dictionary mapping class labels with corresponding emotions
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
angry_value=0
disgusted_value=0
fearful_value=0
happy_value=0
neutral_value=0
sad_value=0
surprised_value=0

#live feed from camera
cap = cv2.VideoCapture(0)

while True:
    # Capture frame
    ret, frame = cap.read()
    if not ret:
        break

    # Find haar cascade to draw bounding box around face
    facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)
  
    #implementing the bounding box + emotion detection text
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x-10, y-40), (x+w, y+h+10), (100, 84, 196), 4)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        prediction = model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        #calculating score for each emotion
        if prediction[0][0] == 1:
            angry_value+=0.01
        if prediction[0][1] == 1:
            disgusted_value+=0.1  
        if prediction[0][2] == 1:
            fearful_value+=0.01 
        if prediction[0][3] == 1:
            happy_value+=0.01
        if prediction[0][4] == 1:
            neutral_value+=0.01  
        if prediction[0][5] == 1:
            sad_value+=0.01
        if prediction[0][6] == 1:
            surprised_value+=0.01
        
    cv2.imshow('Video', cv2.resize(frame,(500,500),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 32ms/step


testing


In [11]:
import time
import matplotlib.pyplot as plt

In [12]:
print(neutral_value)
print(angry_value)
print(sad_value)
print(fearful_value)
print(disgusted_value)
print(surprised_value)
print(happy_value)

1.0200000000000007
0.2800000000000001
0.02
0.07
0.7
0.16
0.2700000000000001
